# Text Mining Project

### NOVA IMS MT Metrics Shared Task

**Group members:**
- Lorenzo Pigozzi	--- m20200745
- Davide Farinati
- Antonio
- Luis Reis

**Objective**

The goal of this project is to develop a metric that predicts the quality of a translation using the reference. 

\
**Evaluation**\
Your metric should correlate well with the existing quality assessments that you have in the above corpus.
The one that we have so far are just the training sets, we will have a test set without the z-score, average score and annotators.
Produce our own metric that will be compared with the existing ones.


<a class="anchor" id="0.1"></a>
# **Table of Contents**

1.	[Importing libraries and corpora](#1)   
2.	[Brief Exploratory data analysis (EDA)](#2)       
3.	[Baseline](#3)     
 3.1. [Pre-processing](#3.1.)\
 3.2. [Feature Extraction](#3.2.)\
 3.3. [Model](#3.3.)\
 3.4. [Evaluation](#3.4.)

**Notes**\
Sklearn:
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

## 1. Importing libraries and corpora <a class="anchor" id="1"></a>

In [1]:
# general libraries
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

# word's preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from nltk.translate.bleu_score import sentence_bleu
from bs4 import BeautifulSoup
import string

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# importing the corpora
cs_en = pd.read_csv('corpus\cs-en\scores.csv')
de_en = pd.read_csv('corpus\de-en\scores.csv')
ru_en = pd.read_csv('corpus\scores_ru-en.csv')
zh_en = pd.read_csv('corpus\zh-en\scores.csv')
en_fi = pd.read_csv('corpus\en-fi\scores.csv')
en_zh = pd.read_csv('corpus\en-zh\scores.csv')

## 2. Brief Exploratory Data Analysis <a class="anchor" id="2"></a>

In [3]:
en_fi.head()

source  \
0  You can turn yourself into a pineapple, a dog ...   
1  Also shot were three men: two 29-year-olds and...   
2  The information is stored at the cash register...   
3  Xinhua says that there were traces of hydrochl...   
4  MacDonald, who was brought on board CBC's comm...   

                                           reference  \
0  Voit muuttaa itsesi ananasta, koirasta tai Roy...   
1  Myös ammuttiin kolme miestä: kaksi 29-vuotiait...   
2  Tiedot tallennetaan kassakoneisiin joka tapauk...   
3  Xinhua kertoo, että Xinyin näytteestä oli sunn...   
4  Voitaisiin kuulla CBD: n kommenttitiimin toimi...   

                                         translation   z-score  avg-score  \
0  Voit muuttaa itsesi ananakseksi, koiraksi tai ... -0.286195      34.20   
1  Myös kolmea miestä ammuttiin: kahta 29-vuotias...  0.547076      58.40   
2  Tiedot kuitenkin tallentuvat kassoilla joka ta...  1.122476      74.60   
3  Xinhua kertoo, että Xinyin sunnuntaina antamas...  0.383095      53.60   
4  MacDonaldin, joka tuli CBC:n selostajatiimiin ... -0.493065      32.25   

   annotators  
0           5  
1           5  
2           5  
3           5  
4           4

**Notes**\
All the the Dataframes imported have 6 columns:
- Sources: the original text in the original language
- Reference: The correct translation
- Translation: the translation that we will evaluate
- Z-score: score of the translation normalized
- Avg-score: score of the translation from 0 to 100
- Annotators: number of annotators

In [4]:
print('Total number of translations : ',len(cs_en) + len(de_en) + len(ru_en) + len(zh_en) + len(en_fi) + len(en_zh) )

Total number of translations :  94657


## 3. Baseline <a class="anchor" id="3"></a>

**Idea**\
Splitting the problem in 3:
- estimate the translations from other languages to English
- estimate the translations from English to Finnish
- estimate the translations from English to Chinese

\
Reason:\
Proably going forward we will need to have different preprocessing for English and for the other 2 languages

### 3.1. Pre-processing <a class="anchor" id="3.1."></a>

In [5]:
cs_en.head()

source  \
0  Uchopíte pak zbraň mezi své předloktí a rameno...   
1  Ale je-li New York změna, pak je to také znovu...   
2  Dlouho a intenzivně jsem během léta přemýšlel,...   
3         Najdou si jiný způsob, jak někde podvádět.   
4  Zpráva o výměně v čele prezidentovy administra...   

                                           reference  \
0  You will then grab the weapon between your for...   
1  But if New York is changed, then it's also a r...   
2  I have been thinking over and over again over ...   
3          They find another way to cheat somewhere.   
4  The report on the replacement of the president...   

                                         translation   z-score  avg-score  \
0  You then grasp the gun between your forearm an... -0.675383  60.000000   
1  But if New York is change, it is also reinvent... -0.829403  44.000000   
2  I have thought long and hard over the course o...  0.803185  96.500000   
3  They will find another way how to defraud others.  0.563149  90.500000   
4  The news of the replacement at the top of the ...  0.021549  74.666667   

   annotators  
0           3  
1           2  
2           2  
3           2  
4           3

In [6]:
# selecting the necessary variables for the baseline
cs_en = cs_en[['reference','translation','avg-score']]
de_en = de_en[['reference','translation','avg-score']]
ru_en = ru_en[['reference','translation','avg-score']]
zh_en = zh_en[['reference','translation','avg-score']]

In [7]:
cs_en.head()

reference  \
0  You will then grab the weapon between your for...   
1  But if New York is changed, then it's also a r...   
2  I have been thinking over and over again over ...   
3          They find another way to cheat somewhere.   
4  The report on the replacement of the president...   

                                         translation  avg-score  
0  You then grasp the gun between your forearm an...  60.000000  
1  But if New York is change, it is also reinvent...  44.000000  
2  I have thought long and hard over the course o...  96.500000  
3  They will find another way how to defraud others.  90.500000  
4  The news of the replacement at the top of the ...  74.666667

In [8]:
# joining the 4 datasets with English translations
df = pd.concat([cs_en, de_en, ru_en, zh_en], axis = 0).reset_index(drop=True)

In [9]:
# a few options for preprocessing
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [10]:
# function to clean the text
def clean(text_list, lemmatize=False):
    """
    Function that a receives a list of strings and preprocesses it.
    
    :param text_list: List of strings.
    :param lemmatize: Tag to apply lemmatization if True.
    :param stemmer: Tag to apply the stemmer if True.
    """
    updates = []
    for j in tqdm(range(len(text_list))):
        
        text = text_list[j]
        
        #LOWERCASE TEXT
        text = text.lower()
        
        #REMOVE NUMERICAL DATA AND PUNCTUATION
        text = re.sub("[^a-z]", ' ', text)
        
        #REMOVE TAGS
        text = BeautifulSoup(text).get_text()
        
        if lemmatize:
            text = " ".join(lemma.lemmatize(word) for word in text.split())
        
        updates.append(text)
        
    return updates

# function to update the dataframe with the cleaned text
def update_df(dataframe, list_updated, variable):
    dataframe.update(pd.DataFrame({variable: list_updated}))

In [11]:
# cleaning the references
updates_reference = clean(df["reference"], lemmatize = True)
update_df(df, updates_reference, "reference")

<ipython-input-10-6ecb4a442cb3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm(range(len(text_list))):


In [12]:
# cleaning the translations
updates_translation = clean(df["translation"], lemmatize = True)
update_df(df, updates_translation, "translation")
df.head()

<ipython-input-10-6ecb4a442cb3>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for j in tqdm(range(len(text_list))):


reference  \
0  you will then grab the weapon between your for...   
1  but if new york is changed then it s also a re...   
2  i have been thinking over and over again over ...   
3           they find another way to cheat somewhere   
4  the report on the replacement of the president...   

                                         translation  avg-score  
0  you then grasp the gun between your forearm an...  60.000000  
1   but if new york is change it is also reinvention  44.000000  
2  i have thought long and hard over the course o...  96.500000  
3   they will find another way how to defraud others  90.500000  
4  the news of the replacement at the top of the ...  74.666667

### 3.2. Train, Development and Test sets <a class="anchor" id="3.2."></a>

In [13]:
## train-dev-test split
train, test = train_test_split(df, test_size=0.2)
test, dev = train_test_split(test, test_size=0.5)

### 3.3. Bag of Words

In [14]:
# # creating Bag-of-Words
# cv = CountVectorizer(max_df=0.9, binary=True)
# # fitting and transforming based on the references
# references = cv.fit_transform(train[['reference', 'translation']])
# # only transforming the translations
# translations = cv.transform(dev[['reference', 'translation']])

In [15]:
baseline_encoder = CountVectorizer()
names = ['train', 'dev', 'test']
for i,df in enumerate([train, dev, test]):
    for column in ['reference', 'translation']:
        encoded_df = names[i] + '_encoded_' + column
        if i == 0:
            vars()[encoded_df] = baseline_encoder.fit_transform(df[column]).todense()
        else:
            vars()[encoded_df] = baseline_encoder.transform(df[column]).todense()
            
    y_name = 'y_' + names[i].split('_')[0]
    vars()[y_name] = np.array(df['avg-score'])

In [16]:
dev_encoded_reference

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### BLEU

**How to evauate a translation: BLEU**\
One thing you might do is look at each word in the output sentence and assign it a score of 1 if it shows up in any of the reference sentences and 0 if it doesn’t. Then, to normalize that count so that it’s always between 0 and 1, you can divide the number of words that showed up in one of the reference translations by the total number of words in the output sentence. This gives us a measure called unigram precision.\
https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213

\
**A Gentle Introduction to Calculating the BLEU Score for Text in Python**
https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

\
**NLP Metrics Made Simple: The BLEU Score**\
https://towardsdatascience.com/nlp-metrics-made-simple-the-bleu-score-b06b14fbdbc1

In [17]:
## Example
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

1.0


In [18]:
# cumulative BLEU score
def avg_bleu_score(df, ngram = 'unigram'):
    bleu_scores = []
    for i in df.index:
        reference = []
        reference.append(df['reference'][i].split(' '))
        translation = df['translation'][i].split(' ')
        if ngram == 'bigram':
            bleu_scores.append(sentence_bleu(reference, translation, weights=(0.5, 0.5, 0, 0)))
        elif ngram == '3-gram':
            bleu_scores.append(sentence_bleu(reference, translation, weights=(0.33, 0.33, 0.33, 0)))
        elif ngram == '4-gram':
            bleu_scores.append(sentence_bleu(reference, translation, weights=(0.25, 0.25, 0.25, 0.25)))
        else:
            bleu_scores.append(sentence_bleu(reference, translation))
    return pd.Series(bleu_scores).mean()

In [19]:
avg_bleu_score(train)

C:\Users\lorep\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\lorep\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\lorep\anaconda3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnin

0.20756861430586115

In [20]:
avg_bleu_score(train, ngram='bigram')

0.40865181478044393

In [24]:
avg_bleu_score(train, ngram='3-gram')

0.296975411534373

In [25]:
avg_bleu_score(train, ngram='4-gram')

0.20756861430586115

In [22]:
# # checking the correlation with the avg-score
# bleu_score_relation = pd.concat([train.reset_index(drop=True), pd.Series(bleu_scores)], axis = 1).iloc[:, -2:]
# bleu_score_relation.head()

In [23]:
# bleu_score_relation.corr(method='pearson')

### 3.3. Model <a class="anchor" id="3.3."></a>

### 3.4. Evaluation <a class="anchor" id="3.4."></a>